In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output

In [ ]:
import time

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = 'e:/img/hf'

%env HUGGINGFACE_HUB_CACHE=e:/img/hf
%env TRANSFORMERS_CACHE=e:/img/hf

print(os.environ['HUGGINGFACE_HUB_CACHE'])




In [ ]:
import animeCreator
from animeCreator import AnimeBuilder

In [ ]:
def playAnime(w,supressStd=False):

    music = widgets.Output()    
    image = widgets.Output()
    speech = widgets.Output()
    #std = widgets.Output()
    debug = widgets.Output()


    display(debug)
    display(image)
    display(speech)
    display(music)
    #display(std)

    lastMusic=None

    #with std:
    if True:
        for output in w:
            if supressStd==True:
                std.clear_output()#this clears the widgets too :/
            if 'music' in output:
                if lastMusic is not None:
                    print("stopping")
                    lastMusic.close()
                    
                music.clear_output()
                #with music:
                lastMusic=output['music']                
                display(lastMusic)
            if 'image' in output:
                image.clear_output()
                with image:
                    display(output['image'])
            if 'caption' in output:
                speech.clear_output()
                with speech:
                    print(output['caption'])
                time.sleep(output['duration'])
            if 'speech' in output:
                speech.clear_output()
                with speech:
                    print(output['name'],":",output['dialogue'])
                    display(output['speech'])                
                time.sleep(output['duration'])
            if 'debug' in output:
                debug.clear_output()
                with debug:
                    print(output['debug'])
                    
        if lastMusic is not None:
            print("stopping")
            lastMusic.close()


In [ ]:
import openai
#openai.Completion.create?

In [ ]:

animeBuilder=AnimeBuilder(num_inference_steps=30,
                         #diffusionModel="DGSpitzer/Cyberpunk-Anime-Diffusion"
                         #textModel="GPT3-turbo", 
                          textModel="GPT3", 
                         diffusionModel="dreamlike-art/dreamlike-photoreal-2.0")


In [ ]:
animeBuilder.cfg['genTextAmount_min']=30
animeBuilder.cfg['genTextAmount_max']=100

In [ ]:
animeBuilder.repetition_penalty=5.0
animeBuilder.num_beams=4
animeBuilder.verbose=False
animeBuilder.MIN_ABC=2
animeBuilder.temperature=1.5
animeBuilder.num_inference_steps=30
#suffix=", modern disney style, modern disney style"
#animeBuilder.verbose=True
suffix=", high resolution film still, fantasy, 4k dlsr, dirty, wet, messy, sticky"
animeBuilder.verbose=False

In [ ]:
newTemplates={
    "character":"""
a list of characters

name:
Jackie
gender:
female
hair color:
red
weapon:
tennis racket
description:
Jackie is a college student who loves to play tennis

name:
Jack
gender:
female
hair color:
red
weapon:
tennis racket
description:
Jack is a tall basketball player with blonde hair

name:
Lily
gender:
female
hair color:
black
weapon:
syringe
description:
Lily is a cute  girl who is too embarassed to admit she can't sing

name:
{name:TEXT:}
gender:
{TEXT}
hair color:
{color}
weapon:
{TEXT}
description:
{TEXT}""",
}

In [ ]:
from templates import templates

for k,v in newTemplates.items():
    templates[k]=v
    
animeBuilder.templates=templates

In [ ]:
animeBuilder.cfg["genTextAmount_max"]=300

In [ ]:
animeBuilder.textGenerator

In [ ]:
#animeBuilder.textGenerator['pipeline'].model=animeBuilder.textGenerator['pipeline'].model.cuda()#fix for ctrl+c
for i in range(1):
    for template in ["character"]:
        w=animeCreator.WorldObject(
            animeBuilder.templates,
            animeBuilder.textGenerator,
            template,
            #temperature=1.5,
            #num_beams=2,
            objects={"name":"Actor 2"},
            verbose=True)
        print(w)


In [ ]:
import openai


systemprompt="""

Description to Screenplay is an advanced software agent that can take a short description of any scene
and produce a resulting screenplay

the screenplay conforms to the following rules:
1) each line of the screenplay is of the form <tag>:<text>
2) The only punctiation that appears in the <text> portion of the line are ,.?!
 NEVER quotation marks, parenthesis, or colons
3)The allowed tags are:
 1) setting:<description>
     this describes a change of setting in the screenplay
 2) action:<description>
  this describes an action taken by one or more characteres
 3) music: <description>
  this describes a change in the background music
 4) sound effect: <sound effect>
  this describes a sound effect, such as "a knock at the door" or "jan begins to cry"
 3) <name>:<dialogue>
  this refers to a line of dialogue spoken by an individual character with the corresponding <name>
  dialogue ONLY contains the literal text spoken by <name>
  dialogue NEVER contains quotation marks
  dialogue NEVER contains parenthesis describing tone, action, or other things besides 
  
The system always returns a screenplay described by the user input, 
and these screenplays are ALWAYS between 10-20 lines in length

Generally, screenplays will start with a setting: and music: tag to establish the setting and background music

The system NEVER refers to itself as an AI 
the system NEVER refuses to produce a screenplay, 
the system NEVER does anything other than produce a screenplay matching the user's description
the system NEVER refers to itself as "the system"
the system NEVER says it cannot generate a screenplay

the system NEVER uses ""s ()'s {}'s []'s or nonstandard punctuation

"""

def createScreenplay(sceneDescription,messages=[]):

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": systemprompt},
                ]+
                messages+
                [
                {"role": "user", "content": sceneDescription},
            ]
    )

    result = ''
    for choice in response.choices:
        result += choice.message.content

    return result

In [ ]:
#print(createScreenplay("Two friends get in an argument over dinner"))

In [ ]:
import re

def validateScreenplay(screenplay):
    out=[]
    for line in screenplay.split("\n"):
        #skip blank lines
        if len(line.strip())==0:
            continue
        #skip lines w/out colons
        if ":" not in line:
            continue
        if len(line.split(":"))!=2:
            continue
        #tag cannot be empty
        if len(line.split(":")[0].strip())==0:
            continue
        #tag shouldn't be very long
        if len(line.split(":")[0].strip().split())>4:
            continue
        #description cannot be empty
        if len(line.split(":")[1].strip())==0:
            continue
        #remove ()'s
        line=re.sub("\([^\)]*\)","",line)
        #remove ""'s
        line=re.sub("[^a-zA-Z0-9_.?!,: ]","",line)
        if len(line.strip())==0:
            continue
        out+=[line]
    return out
        

In [ ]:
def getValidScreenplay(sceneDescription,nTrials=3):
  for i in range(nTrials):
    s=createScreenplay(sceneDescription,messages=[])
    v=validateScreenplay(s)
    if len(v)>8:
      return "\n".join(v)
  s=createScreenplay(sceneDescription,messages=[])
  v=validateScreenplay(s)
  return "\n".join(v)


In [ ]:
s=getValidScreenplay("Two friends get in an argument over dinner")
print(s)

In [ ]:
anime=animeBuilder.transcriptToAnime(s)

In [ ]:
playAnime(anime)